In [30]:
# Checks if n is a power and returns whatever it is a power of
def isPower(n):
    if n == 0:
        return [True, 0]
    if n == 1:
        return [True, 1]
    if n < 4:
        return [False, 0]
    for i in range(int(math.log(n,2)), 1, -1):
        if pow(n,1/i).is_integer():
            return [True, pow(n,1/i)]
    return [False, 0]

In [49]:
# Performs fast addition to evaluating [k]P by multiplying P by 2 each time and
# adding that result if the binary representation of k has a 1 in that spot
def fast_addition(E, P, k, verbose):
    out = E([0, 1, 0])
    r = int(log(k, 2))
    if k % 2 == 1:
        out += P
    for i in range(1, r + 1):
        try:
            P = P + P
        except ZeroDivisionError as err:
            errStr = "Error: {0}".format(err)
            if verbose: print( ' '.join(errStr.split(' ')[:7]) )
            return(int(errStr.split(' ')[3]))
            break
        except AttributeError as err:
            if verbose: print("other error: KeyError")
        except AttributeError as err:
            if verbose: print("other error: AttributeError")
        if int((k >> i) & 1) == 1:
            try:
                out += P
            except ZeroDivisionError as err:
                errStr = "Error: {0}".format(err)
                if verbose: print( ' '.join(errStr.split(' ')[:7]) )
                return(int(errStr.split(' ')[3]))
                break
            except AttributeError as err:
                if verbose: print("other error: KeyError")
            except AttributeError as err:
                if verbose: print("other error: AttributeError")
    return(out)

In [89]:
COUNT_G_N = 10
COUNT_TOTAL = 50
COUNT_SAME_B = 10
# Uses EC Factorization to find a single factor of n
def find_factor_please(n, verbose):
    if verbose: print("Factoring " + str(n) + "...")
    if (is_prime(int(n))):
        if verbose: print(str(n) + " is prime...")
        return n
    B= int(math.e^( 1/2 * sqrt(log(n) * log(log(n)) ) )) + 1
    if isPower(n)[0]:
        return isPower(n)[1]
    counter_g_n = 0
    count_same_B = 0
    count_total = 0
    while count_total < COUNT_TOTAL:
        count_same_B = 0
        if verbose: print("###########################")
        M = 2
        for i in range(3, B + 1):
            M = lcm(M, i)
        while count_same_B < COUNT_SAME_B and counter_g_n < COUNT_G_N :
            if verbose: print(".......................")
            if verbose: print("Count gcd = n: " + str(counter_g_n))
            a = randint(1, n - 1)
            x = randint(1, n - 1)
            y = randint(1, n - 1)
            b = y^2 - x^3 - a*x
            g = gcd(4*a^3 + 27*b^2, n)
            if (1 < g < n):
                return gcd(4*a^3 + 27*b^2, n)
            elif g == n:
                continue
            E = EllipticCurve(IntegerModRing(n),[a,b]);
            P = E([x, y ,1])
            if verbose: print(E, ",  P=",P)
            Q = fast_addition(E, P, M, verbose)
            if isinstance(Q, int):
                if gcd(Q,n) != n:
                    if verbose: print("Found GCD: " + str(gcd(Q,n)))
                    return(gcd(Q,n))
                else:
                    counter_g_n += 1
            else:
                if verbose: print("No errors in calculating [M]P")
            count_same_B += 1
        if counter_g_n >= COUNT_G_N:
            B = int(sqrt(B))
            counter_g_n = 0
        else:
            B *= 2
        count_total += 1
    return 0

In [84]:
# Finds all the factors of n by repeatedly calling the find_factor_please method
# until all factors are prime
def factor_please(n, verbose):
    n_original = n
    factors = []
    while (int(n) & 1 == 0):
        if verbose: print("2 is a factor...")
        factors.append(2)
        n = n/2
    while (n % 3 == 0):
        if verbose: print("3 is a factor...")
        factors.append(3)
        n = n/3
    while (not is_prime(int(n)) and n != 1):
        f = find_factor_please(n, verbose)
        if f == 0:
            return [0]
        if not is_prime(f):
            factors_f = factor_please(int(f), verbose)
            for factor in factors_f:
                factors.append(factor)
        else:
            factors.append(int(f))
        n = int(n/f)
    if n!= 1: factors.append(int(n))
    return factors

In [85]:
# Tidies up a list of jumbled factors (expected output of factor_please) and tidies them
# up in increasing order to their respective powers
def tidy_factors(n, factors, print_output):
    factors.sort()
    if factors == [0]:
        return "Error..."
    powered_factors = []
    current_factor = factors[0]
    factor_string = ""
    count = 1
    for i in range(1, len(factors)):
        if factors[i] == current_factor:
            count += 1
        else:
            powered_factors.append([current_factor, count])
            if print_output:
                factor_string += str(current_factor)
                if count > 1:
                    factor_string += '^' + str(count)
                factor_string += ' * '
            count = 1
        current_factor = factors[i]
    powered_factors.append([current_factor, count])
    if print_output:
        factor_string += str(current_factor)
        if count > 1:
            factor_string += '^' + str(count)
    if print_output: print(str(n) + " = " + factor_string)
    return powered_factors

In [87]:
# Returns the complete list of tuples of factors and their respective powers
# and prints that out if desired
def tidy_factor_please(n, verbose, print_output):
    factors = factor_please(n, verbose)
    return tidy_factors(n, factors, print_output)

In [86]:
n = 45687653345678754
print("n = " + str(n))
factors_n_my_func = tidy_factor_please(n, False, True);
print(str(n) + " = " + str(factor(n)))


n = 45687653345678754
45687653345678754 = 2 * 3 * 73 * 414889 * 251415947
45687653345678754 = 2 * 3 * 73 * 414889 * 251415947


In [59]:
n = 85643458435^4
print("n = " + str(n))
factors_n_my_func = tidy_factor_please(n, False, True);
print(str(n) + " = " + str(factor(n)))

n = 53799319978834899310238247597331284893100625


53799319978834899310238247597331284893100625 = 5^4 * 57073^4 * 300119^4
53799319978834899310238247597331284893100625 = 5^4 * 57073^4 * 300119^4


In [90]:
n = 4567834563080237845679179458328743598
print("n = " + str(n))
factors_n_my_func = tidy_factor_please(n, True, True);
print(str(n) + " = " + str(factor(n)))

n = 4567834563080237845679179458328743598
2 is a factor...
Factoring 2283917281540118922839589729164371799...
###########################
.......................
Count gcd = n: 0
Elliptic Curve defined by y^2 = x^3 + 1187303310643976120027979782240245225*x + 1038626680306660582720166512387670262 over Ring of integers modulo 2283917281540118922839589729164371799 

,  P= (1688657541001188658958175705528307030 : 1136716493987702719717539110379343916 : 1)


No errors in calculating [M]P
.......................
Count gcd = n: 0
Elliptic Curve defined by y^2 = x^3 + 2190549657827936260054252672650056629*x + 2012785557481574501323627828618033727 over Ring of integers modulo 2283917281540118922839589729164371799 ,  P= (436650599622584133374483039310229685 : 1071167278046525183033088973404918963 : 1)


No errors in calculating [M]P
.......................
Count gcd = n: 0
Elliptic Curve defined by y^2 = x^3 + 576920662352772229736881224862304513*x + 1284922268546650097118992918947480085 over Ring of integers modulo 2283917281540118922839589729164371799 ,  P= (1816316524656607928751230621013820203 : 802295194329208765403543037296637681 : 1)


No errors in calculating [M]P
.......................
Count gcd = n: 0
Elliptic Curve defined by y^2 = x^3 + 367327100928418681305973900230169123*x + 785215509908490447516092228740525473 over Ring of integers modulo 2283917281540118922839589729164371799 ,  P= (1877677447946645826033789579497141385 : 1275057078242463972680131612640672421 : 1)


Error: Inverse of 861857941902767090670861338748615058 does not exist
Found GCD: 184169457253
4567834563080237845679179458328743598 = 2 * 184169457253 * 12401172895908696631356683
4567834563080237845679179458328743598 = 2 * 184169457253 * 12401172895908696631356683


In [76]:
p = random_prime(10^10, proof = True)
print(p)
q = random_prime(10^3, proof = True)
print(q)
r = random_prime(10^12, proof = True)
print(r)
n = p^2*q^3*r
print("n = " + str(n))
factors_n_my_func = tidy_factor_please(n, False, True);
print(str(n) + " = " + str(factor(n)))

8159292649
269
689509529543
n = 893515587635422634690654443566447971587


893515587635422634690654443566447971587 = 269^3 * 8159292649^2 * 689509529543
893515587635422634690654443566447971587 = 269^3 * 8159292649^2 * 689509529543


In [83]:
p = random_prime(10^10, proof = True)
print(p)
q = random_prime(10^3, proof = True)
print(q)
r = random_prime(10^12, proof = True)
print(r)
s = random_prime(10^10, proof = True)
print(s)
n = p*q^3*r*s
print("n = " + str(n))
factors_n_my_func = tidy_factor_please(n, False, True);
print(str(n) + " = " + str(factor(n)))

2310689267
479
295322068813
9404448511
n = 705305584414931337926327561939791615159


705305584414931337926327561939791615159 = 479^3 * 2310689267 * 9404448511 * 295322068813
705305584414931337926327561939791615159 = 479^3 * 2310689267 * 9404448511 * 295322068813


In [3]:
a = randint(1, 29 - 1)
x = randint(1, 29 - 1)
y = randint(1, 29 - 1)
b = y^2 - x^3 - a*x
E = EllipticCurve(GF(29),[a,b]);
P = E([x, y ,1])
print(E.order())

30
